In [18]:
from datasets import load_dataset

In [19]:
# Загрузка исходного датасета
dataset = load_dataset("kumarharsh1812/test")

# Выбор нужных колонок
selected_dataset = dataset['train'].select_columns(["test_scenario", "test_steps"])

# Разделение на тренировочную и тестовую выборки (80/20)
split_dataset = selected_dataset.train_test_split(test_size=0.2, seed=42)

In [7]:
split_dataset['train'][:3]

{'test_scenario': ['Verify that a user can successfully modify their subscription email terms on the sports and fitness equipment website.',
  'Verify that the streaming quality is consistent during a live concert.',
  'Verify the system behavior when scanning an invalid or expired ticket at event check-in.'],
 'test_steps': ['1. Login to the sports and fitness equipment website.\n2. Navigate to the subscription settings section.\n3. Modify the email terms for subscription.\n4. Save the changes.',
  '1. Access a live concert streaming.\n2. Observe the video and audio quality throughout the concert.',
  '1. Scan an invalid or expired ticket QR code at the event check-in.\n2. Verify that the system prompts an error message indicating the ticket is invalid or expired.']}

In [8]:
split_dataset['test'][:3]

{'test_scenario': ['Verify course progress sync across devices when there are no interruptions.',
  'Validate the accuracy of QoS metrics monitoring tool by monitoring network latency during peak hours.',
  'Verify that a user can send a message to a project collaborator.'],
 'test_steps': ['1. Login to the online learning platform on Device A and start a course.\n2. Verify that the progress is synced to Device B without any interruptions.',
  '1. Simulate peak hours traffic on the network.\n2. Monitor network latency for video streaming.',
  '1. Log in to the platform as a project collaborator.\n2. Navigate to the messaging feature.\n3. Select a project collaborator to send a message.\n4. Type a message and click on the send button.']}

In [22]:
# Сохранение полученного датасета в локальную директорию 'dataset'
split_dataset.save_to_disk('dataset')

Saving the dataset (1/1 shards): 100%|██████████| 935/935 [00:00<00:00, 233766.94 examples/s]
